In [1]:
!python --version

Python 3.12.12


In [2]:
with open('/content/wizard_of_oz.txt','r',encoding='utf-8') as f:
  text = f.read()
print(text[:200])

﻿  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [3]:
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [5]:
encode('hello')

[61, 58, 65, 65, 68]

In [6]:
decode([44,55,66])

'Tbm'

In [7]:
import torch

In [8]:
data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [9]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[:n]

In [10]:
block_size = 8
batch_size = 4

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'When input is : {context} --> target is {target}')

When input is : tensor([80]) --> target is 1
When input is : tensor([80,  1]) --> target is 1
When input is : tensor([80,  1,  1]) --> target is 28
When input is : tensor([80,  1,  1, 28]) --> target is 39
When input is : tensor([80,  1,  1, 28, 39]) --> target is 42
When input is : tensor([80,  1,  1, 28, 39, 42]) --> target is 39
When input is : tensor([80,  1,  1, 28, 39, 42, 39]) --> target is 44
When input is : tensor([80,  1,  1, 28, 39, 42, 39, 44]) --> target is 32


The langauge models needs to be scaled to process and handle large number of input data without affecting its processing time and performance. <br>
Here , `batch_size` & `block_size` helps in fast training and good performance.

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'CPU'
print(device)


CPU


In [13]:
input = torch.empty(2,3)
input

tensor([[1.6420e-38, 0.0000e+00, 1.6420e-38],
        [0.0000e+00, 1.6415e-38, 0.0000e+00]])

### To compare the difference in processing time between CPU and GPU

In [14]:
'''
import time
import numpy as np

torch_rand1 = torch.rand(100,100,100,100).to(device)
torch_tand2 = torch.rand(100,100,100,100).to(device)
np_rand1 = torch.rand(100,100,100,100)
np_rand2 = torch.rand(100,100,100,100)

start_time = time.time()
rand = (torch_rand1 @ torch_tand2)
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Elapsed GPU Processing time : {elapsed_time}')

start_time = time.time()
rand = np.multiply(np_rand1,np_rand2)
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Elapsed CPU Processing time : {elapsed_time}')
'''

"\nimport time\nimport numpy as np\n\ntorch_rand1 = torch.rand(100,100,100,100).to(device)\ntorch_tand2 = torch.rand(100,100,100,100).to(device)\nnp_rand1 = torch.rand(100,100,100,100)\nnp_rand2 = torch.rand(100,100,100,100)\n\nstart_time = time.time()\nrand = (torch_rand1 @ torch_tand2)\nend_time = time.time()\nelapsed_time = end_time - start_time\nprint(f'Elapsed GPU Processing time : {elapsed_time}')\n\nstart_time = time.time()\nrand = np.multiply(np_rand1,np_rand2)\nend_time = time.time()\nelapsed_time = end_time - start_time\nprint(f'Elapsed CPU Processing time : {elapsed_time}')\n"

In [15]:
### Let's do a recap on torch and its function
prob = torch.tensor([0.1,0.9]) #Prob Tensor

# 10% or 0.1 --> 0 , 90% or 0.9 --> 1. Each prob points to the probability of that index in the tensor
# Draw 5 samples
samples = torch.multinomial(prob,num_samples=10,replacement=True)
print(samples)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [16]:
# Torch concat
tensor = torch.tensor([1,2,3,4,5])
op = torch.cat((tensor,torch.tensor([6,7,8,9])),dim=0)
print(op)

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])


In [17]:
# tril - traingular lower
out = torch.tril(torch.ones(5,5))
out

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [18]:
a = torch.tensor([[1., 2., 3.],
                  [4., 5., 6.],
                  [7., 8., 9.]])
trildef = torch.tril(a)
print(trildef)

tensor([[1., 0., 0.],
        [4., 5., 0.],
        [7., 8., 9.]])


In [19]:
trilpos = torch.tril(a,diagonal=1)
print(trilpos)

tensor([[1., 2., 0.],
        [4., 5., 6.],
        [7., 8., 9.]])


In [20]:
trilneg = torch.tril(a,diagonal=-1)
print(trilneg)

tensor([[0., 0., 0.],
        [4., 0., 0.],
        [7., 8., 0.]])


In [21]:
out = torch.triu(torch.ones(5,5))
out

tensor([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]])

In [24]:
out = torch.zeros(5,5).masked_fill(torch.tril(torch.ones(5,5))==0,float('-inf'))
out

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])

In [25]:
torch.exp(out)

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

That tensor is a Causal Attention Mask, crucial in LLMs (like GPT) for autoregressive text generation, ensuring the model predicts the next word by only looking at past words, not future ones, by setting future attention scores to negative infinity (which softmax turns to zero), preventing "cheating" and enabling sequential prediction. Its purpose is to enforce causality, simulating how humans read and write one word at a time, making the LLM generate coherent, forward-flowing text.
<br>

Why it's Important (Purpose):<br>
- Autoregressive Generation: LLMs generate text token by token (word by word). To predict the next token, the model needs context from the previous tokens. This mask ensures it doesn't see future tokens, forcing it to learn dependencies correctly.
Prevents Information Leakage: Without the mask, a token could "look ahead" at the answer, which would make training easy but useless for real-world generation where future context isn't available.
- Enforces Sequential Learning: It mimics reading from left to right (or right to left, depending on direction), allowing the model to build understanding incrementally.
How it Works (Mechanism)
Attention Scores: In self-attention, a score is calculated for how much each token should attend to every other token.
Mask Application: The mask (your tensor) is added to these scores before the softmax function.
- Softmax Effect:
e^(-∞) (e to the power of negative infinity) is effectively zero.
When the mask applies -inf to future positions, their attention scores become zero after softmax.
This means those future tokens get zero probability and don't influence the current token's representation. <br>

Example Breakdown (Your Tensor):<br>
- tensor([[0., "-inf," "-inf," "-inf," "-inf], - The first token (index 0) can only attend to itself (the 0.), not future ones.
- " [0., 0., "-inf," "-inf," "-inf], - The second token (index 1) can attend to itself and the first, but not the third, fourth, or fifth.
- ...and so on, creating a lower triangular matrix of allowed attention, allowing only past and current context.

In [26]:
input = torch.zeros(2,3,4)
out = input.transpose(0,2)
out.shape

torch.Size([4, 3, 2])

In [27]:
t1 = torch.tensor([1,2,3])
t2 = torch.tensor([4,5,6])
t3 = torch.tensor([7,8,9])

stack = torch.stack([t1,t2,t3])
stack

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [32]:
import torch.nn as nn
sample = torch.tensor([10.,10.,10.])
linear = nn.Linear(3,3,bias=False)
print(linear(sample))

tensor([-3.9351, -5.9054, -5.1283], grad_fn=<SqueezeBackward4>)


In [33]:
import torch.nn.functional as F

t1 = torch.tensor([1.0,2.0,3.0])
softmaxop = F.softmax(t1,dim=0)

print(softmaxop)

tensor([0.0900, 0.2447, 0.6652])


In [34]:
a = torch.tensor([[1,2],[3,4],[5,6]])
b = torch.tensor([[7,8,9],[10,11,12]])

print(torch.matmul(a,b))

tensor([[ 27,  30,  33],
        [ 61,  68,  75],
        [ 95, 106, 117]])


In [35]:
int64 = torch.randint(1,(3,2))
float32 = torch.rand(2,3)

result = torch.matmul(int64,float32)
print(result)

RuntimeError: expected m1 and m2 to have the same dtype, but got: long int != float